## MachineLearningEngine Class

The MachineLearningEngine class is builds on the CoreEngine class. The CoreEngine class serves as a parent class engines that focus on data, while the MachineLearningEngine class is for engines that focus on learning from data.

In [12]:
from src.StreamPort.ml.MachineLearningEngine import MachineLearningEngine

#Creates an empty MachineLearningEngine object and prints it
engine = MachineLearningEngine()
engine.print()


MachineLearningEngine 
  name: None 
  author: None 
  path: None 
  date: 2024-04-18 17:28:44.620285 



## SensorAnalysis Class

The SensorAnalysis class is builds on the class Analysis. 

In [11]:
from src.StreamPort.ml.SensorAnalysis import SensorAnalysis

engine = SensorAnalysis()
engine.print()

ModuleNotFoundError: No module named 'StreamPort'